In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
import mlflow

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-expe/mlruns/1', creation_time=1747637206697, experiment_id='1', last_update_time=1747637206697, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename).head(90000)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train =read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet')

In [6]:
len(df_train), len(df_val)

(87795, 88147)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

26.665223468577214

In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [12]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet")
    mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [13]:
import xgboost as xgb

In [14]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [15]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=500,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [05:18:05] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.40907                           
[1]	validation-rmse:5.09347                           
[2]	validation-rmse:5.04788                           
[3]	validation-rmse:5.04060                           
[4]	validation-rmse:5.03507                           
[5]	validation-rmse:5.03211                           
[6]	validation-rmse:5.03097                           
[7]	validation-rmse:5.02494                           
[8]	validation-rmse:5.02538                           
[9]	validation-rmse:5.02377                           
[10]	validation-rmse:5.02434                          
[11]	validation-rmse:5.02275                          
[12]	validation-rmse:5.02128                          
[13]	validation-rmse:5.02333                          
[14]	validation-rmse:5.02417                          
[15]	validation-rmse:5.02417                          
[16]	validation-rmse:5.02637                          
[17]	validation-rmse:5.02879                          
[18]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [05:18:12] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.66718                                                     
[1]	validation-rmse:5.29468                                                     
[2]	validation-rmse:5.24558                                                     
[3]	validation-rmse:5.24430                                                     
[4]	validation-rmse:5.24362                                                     
[5]	validation-rmse:5.24218                                                     
[6]	validation-rmse:5.24216                                                     
[7]	validation-rmse:5.24573                                                     
[8]	validation-rmse:5.24744                                                     
[9]	validation-rmse:5.25012                                                     
[10]	validation-rmse:5.25514                                                    
[11]	validation-rmse:5.25575                                                    
[12]	validation-rmse:5.26034

...

[0]	validation-rmse:9.17700                                                     
[1]	validation-rmse:8.76034                                                     
[2]	validation-rmse:8.38464                                                     
[3]	validation-rmse:8.04512                                                     
[4]	validation-rmse:7.74065                                                     
[5]	validation-rmse:7.46462                                                     
[6]	validation-rmse:7.21930                                                     
[7]	validation-rmse:6.99830                                                     
[8]	validation-rmse:6.80099                                                     
[9]	validation-rmse:6.62521                                                     
[10]	validation-rmse:6.46948                                                    
[11]	validation-rmse:6.32868                                                    
[12]	validation-rmse:6.20454

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [05:24:42] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.79810                                                     
[1]	validation-rmse:5.71734                                                     
[2]	validation-rmse:5.32850                                                     
[3]	validation-rmse:5.18692                                                     
[4]	validation-rmse:5.13139                                                     
[5]	validation-rmse:5.10793                                                     
[6]	validation-rmse:5.09600                                                     
[7]	validation-rmse:5.09710                                                     
[8]	validation-rmse:5.09386                                                     
[9]	validation-rmse:5.09174                                                     
[10]	validation-rmse:5.09188                                                    
[11]	validation-rmse:5.09022                                                    
[12]	validation-rmse:5.08796

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [05:24:57] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.28374                                                     
[1]	validation-rmse:6.09523                                                     
[2]	validation-rmse:5.53449                                                     
[3]	validation-rmse:5.26957                                                     
[4]	validation-rmse:5.14636                                                     
[5]	validation-rmse:5.07889                                                     
[6]	validation-rmse:5.04190                                                     
[7]	validation-rmse:5.02288                                                     
[8]	validation-rmse:5.01181                                                     
[9]	validation-rmse:5.00544                                                     
[10]	validation-rmse:5.00193                                                    
[11]	validation-rmse:5.00187                                                    
[12]	validation-rmse:5.00125

In [ ]:
# params = {
#     'learning_rate': 0.3898954493981843,
#     'max_depth': 89,
#     'min_child_weight': 8.462681965760824,
#     'objective': 'reg:linear',
#     'reg_alpha': 0.010629895421629038,
#     'reg_lambda': 0.01859166455098889,
#     'seed': 42
# }

# # mlflow.xgboost.autolog()

# with mlflow.start_run():
#     mlflow.set_tag("model", "xgboost")
#     mlflow.log_param("params",params)
#     booster = xgb.train(
#         params=params,
#         dtrain=train,
#         num_boost_round=500,
#         evals=[(valid, 'validation')],
#         early_stopping_rounds=10
# )

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [05:29:55] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:7.28374
[1]	validation-rmse:6.09523
[2]	validation-rmse:5.53449
[3]	validation-rmse:5.26957
[4]	validation-rmse:5.14636
[5]	validation-rmse:5.07889
[6]	validation-rmse:5.04190
[7]	validation-rmse:5.02288
[8]	validation-rmse:5.01181
[9]	validation-rmse:5.00544
[10]	validation-rmse:5.00193
[11]	validation-rmse:5.00187
[12]	validation-rmse:5.00125
[13]	validation-rmse:5.00188
[14]	validation-rmse:4.99984
[15]	validation-rmse:4.99856
[16]	validation-rmse:4.99805
[17]	validation-rmse:4.99888
[18]	validation-rmse:4.99899
[19]	validation-rmse:5.00042
[20]	validation-rmse:5.00175
[21]	validation-rmse:5.00084
[22]	validation-rmse:5.00283
[23]	validation-rmse:5.00353
[24]	validation-rmse:5.00469
[25]	validation-rmse:5.00368
[26]	validation-rmse:5.00302
